# Gelbenzeiten

In [32]:
import pandas as pd
import json

In [55]:
with open("fixedCountriesToCities.json", 'r') as file:
    binary = file.read()
#     json.loads(binary)
    
# world_cities = pd.read_csv(, encoding="UTF-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 1742: invalid continuation byte

In [53]:
binary[2:100]

b'{\x00"\x00C\x00h\x00i\x00n\x00a\x00"\x00:\x00[\x00"\x00G\x00u\x00a\x00n\x00g\x00z\x00h\x00o\x00u\x00"\x00,\x00"\x00F\x00u\x00z\x00h\x00o\x00u\x00"\x00,\x00"\x00B\x00e\x00i\x00j\x00i\x00n\x00g\x00"\x00,\x00"\x00B\x00a\x00o\x00t\x00o\x00u\x00"\x00'

In [26]:
!wget https://github.com/David-Haim/CountriesToCitiesJSON/blob/master/countriesToCities.json

--2016-08-29 12:57:51--  https://github.com/David-Haim/CountriesToCitiesJSON/blob/master/countriesToCities.json
Resolving github.com... 192.30.253.112
Connecting to github.com|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘countriesToCities.json’

countriesToCities.j     [ <=>                ]  31.54K  --.-KB/s    in 0.1s    

2016-08-29 12:57:51 (332 KB/s) - ‘countriesToCities.json’ saved [32302]

